<a href="https://colab.research.google.com/github/amulyamarali/student_performance/blob/main/Student_Performance.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Artificial Neural Network
### my first artificial brain

# ANN using CLASSIFICATION


### Importing the libraries

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf

In [2]:
tf.__version__

'2.8.2'

## Part 1 - Data Preprocessing

### Importing the dataset

In [3]:
dataset = pd.read_csv("student-por.csv")

Creating a subset of the data set


In [4]:
data2 = dataset[["G1", "G2", "G3", "failures", "studytime", "freetime", "absences", "internet"]].copy()
data2.head()

,G1,G2,G3,failures,studytime,freetime,absences,internet
0,0,11,11,0,2,3,4,no
1,9,11,11,0,2,3,2,yes
2,12,13,12,0,2,3,6,yes
3,14,14,14,0,3,2,0,yes
4,11,13,13,0,2,3,0,no


In [5]:
# as rollno. customerId and surname has no impact on predicting whether the customer leaves or not 
# we will get rid of those columns 
x = data2.iloc[:, 1:].values
y = data2.iloc[:, 0].values

In [6]:
print(x) # printing matrix of features 

[[11 11 0 ... 3 4 'no']
 [11 11 0 ... 3 2 'yes']
 [13 12 0 ... 3 6 'yes']
 ...
 [12 9 0 ... 1 6 'no']
 [10 10 0 ... 4 6 'yes']
 [11 11 0 ... 4 4 'yes']]


In [7]:
print(y) # printing the dependent variable's data 

[ 0  9 12 14 11 12 13 10 15 12 14 10 12 12 14 17 13 13  8 12 12 11 12 10
 10 10 11 11 12 12 10 15 13 13 12 11 14 13 11 14 11 10 14  9 10 10 13 17
 11 13 14 16 10 13 13 12 15 15 14 16 17 10 13 14 13 16 11 10 11 15 13 11
 13 13 11 11 12 13  9 12 11 10 12 13 13 12 13 15 12  9  9 14 12 13 11 13
  9 13 12 12  9 16 12 11 16 10 10 13 12 15 13 11  8 18 10 16 16 14 12 14
 14 14 14 12 12  9 10  9 14 13 10 10 11 11 13 15  9 13 12 13 10 14 11 12
 12  9 10 13  8  9  9 15 12 13 10  9 12 11 11 13 11  9 12 11  9 13 11 13
  7  9 11 12 10  9  8  7  8  9  8  8  9 17 12 16  9 16 11 14 14 10 13 11
 10 11 11 11 17 14 14 11 10 13 12  9 12 12 10 11 13 14 13 10 14 11 14 11
 14 13 13  8 11 14 12 11 12 13 13 12 12 14 11 10 12 12 11 12 13 10 15  8
 17 10  9 15 14 12 13 13  9 12 16  9 14 10  9  8  7 10 14 11 12 13  9  8
 11 15 15 15 12 14 14 11 13 11 10 12 15 11  9  7 10 11  7  9  7 14 12  9
 12 14 10 11 11 11 11 11 16  9 10 10 15 10 10 15 11  8 15 11 10 10  8 11
 16 13 16 14 15 12 12 11 14 13  9 10 10 11 13 17 12

### Encoding categorical data

Label Encoding the "Internet" column

In [8]:
# there are two categories one is the country which is mainly divided into france spian and germany
# the other is gender either female or male 
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
 # if we do x = le.fit_transform(x)  ------> this will apply categorization to all the columns os the dataset
x[:, 6] = le.fit_transform(x[:, 6])  # this will transform only the gender to categories

In [9]:
print(x) # we can see that female is encoded to 0 and male to 1

[[11 11 0 ... 3 4 0]
 [11 11 0 ... 3 2 1]
 [13 12 0 ... 3 6 1]
 ...
 [12 9 0 ... 1 6 0]
 [10 10 0 ... 4 6 1]
 [11 11 0 ... 4 4 1]]


### Splitting the dataset into the Training set and Test set

In [10]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 0)

### Feature Scaling

In [11]:
# feature scaling is extremely IMPORTANT in DEEP LEARNING
# feature scaling is so fundamental that we apply it to all the fetures of the dataset 
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
x_train = sc.fit_transform(x_train) # here we r not selecting any specific features as we are applying for all the columns
x_test = sc.transform(x_test)

In [12]:
x_train

array([[ 0.16090933,  0.04541223, -0.39303124, ...,  0.77849049,
         0.03087478,  0.55138018],
       [-1.87596729, -1.44629329,  1.23883446, ...,  0.77849049,
        -0.81249421,  0.55138018],
       [ 1.51882708,  1.53711774, -0.39303124, ...,  0.77849049,
         0.03087478,  0.55138018],
       ...,
       [-0.85752898, -0.25292887, -0.39303124, ..., -0.18121631,
         0.87424377,  0.55138018],
       [-1.19700842, -0.84961108, -0.39303124, ...,  1.7381973 ,
         0.03087478,  0.55138018],
       [ 0.16090933,  0.34375333,  1.23883446, ..., -0.18121631,
        -0.81249421, -1.81363067]])

## Part 2 - Building the ANN

### Initializing the ANN

In [13]:
# first is to initialize ANN as sequence of layers
# create a variable which will be the ANN itself
# this variable will be created as an object of a certaon class and this class will be a sequential class
# this class will help in building an ANN and a sequence of layers -- input layer, hidden layer and the output layer 
# the other type of Neural Network is a Computational Graph where the neurons connected in any way and NOT in sequential layers 
# examples of such NN are RESTRICTED BULLSMAN MACHINES OR DEEP BULLSMAN MACHINE
ann = tf.keras.models.Sequential() # ann is an object of the Sequential class 
# Sequential is a class taken from the Keras library which is also part of Tensorflow
# ann is an instance of the sequenial class

### Adding the input layer and the first hidden layer

In [14]:
# dense class is used to build a fully connected network at any point in the NN 
ann.add(tf.keras.layers.Dense(units = 6, activation = "relu"))
# add is a mehod in Sequential class
# we will be adding a fully connected layer using the add method 
# this add method will be an instance(object) of the dense class
# units in the Dense class corresponds to number of hidden neurons in the NN
# there is no specific rule in finding the number of hidden neurons needed 
# it has to be found out by experimenting 
# acivation function in a fully connected neural network has to be the Rectifier function as we saw in the lectures
# "relu" is the keyword for the rectifier function


### Adding the second hidden layer

In [15]:
# as add() can add a hidden layer at any stage of the construction layer of a NN
# we can use it to create another hidden layer 
# we can change the number of neurons here but lets keep it the same 
ann.add(tf.keras.layers.Dense(units = 6, activation = "relu"))

### Adding the output layer

In [16]:
# add() method can be used to add any layer including the output layer 
# as we want out output layer to be connected to the second hidden layer we are using the dense class
ann.add(tf.keras.layers.Dense(units = 1, activation = "linear"))

# if the classification is not binary then the classification has to be 'softmax'

# now the number of units in this object depends on the dimension of the dependent variable
# in our case the output for the dependent layer is either 1 or 0
# so the dimension is one
# but if the output was supposed to be either a, b or c then it would got converted into 111 101 100 something like that and it would have 3 dimensions
# so now in our case the units is 1 as we need only 1 neuron

# the activation function is a sigmoid activation function 
# we choose this activation function for the output layer as it not only gives the output but it also predicts the probability for that output to occur


## Part 3 - Training the ANN

### Compiling the ANN

In [17]:
# use the same method object and compile it using the compile method
# optimizer , loss and the matrics are three parameters that go inside the compiler method 

opt = tf.keras.optimizers.RMSprop(0.001)

ann.compile(optimizer=opt, loss = 'mse', metrics=['mae','mse','mape'])

# optimizer helps in adjusting the weights and computing the cost function using the SDG method
# adam optimizer is used so that it can reduce the loss in rhe next iteration 

# when we are doing classification involving binary then we have to use a specific loss called as 'binary_crossentropy'
# if non-bianry classification is done then we have to enter loss as 'categorical_crossentropy'
# In the case of regression problems where a quantity is predicted, it is common to use the mean squared error (MSE) loss function instead.


# metrics is used to train the NN on the different bases 
# for now it is just accuracy

### Training the ANN on the Training set

In [18]:
ann.fit(x_train, y_train, batch_size = 32, epochs = 300)

# fit method is the method used to train the NN
# infact fit method is used to train all the machine learning models

# as batch learning is always efficient when training an ANN 
# instead of comparing the predictions to the real results one by one , compute and reduce the loss
# we instead compare severeal predictions to several real results by making batches
# so the batch_size gives the no. of batches that u will make to compare predicted to real results
# by default we chose 32 but can chose any depending on experimenting  

# a NN has to be trained based the number of EPOCHS to increase the accuracy over time
# we can consider epochs as the no.of iterations the NN will take to learn properly and adopt the best fit weight 

Epoch 1/300
17/17 [==============================] - 1s 3ms/step - loss: 144.9176 - mae: 11.7775 - mse: 144.9176 - mape: 35081.4102
Epoch 2/300
17/17 [==============================] - 0s 2ms/step - loss: 140.7870 - mae: 11.5902 - mse: 140.7870 - mape: 168597.9844
Epoch 3/300
17/17 [==============================] - 0s 2ms/step - loss: 137.3740 - mae: 11.4310 - mse: 137.3740 - mape: 188910.6562
Epoch 4/300
17/17 [==============================] - 0s 2ms/step - loss: 134.0764 - mae: 11.2748 - mse: 134.0764 - mape: 409279.4375
Epoch 5/300
17/17 [==============================] - 0s 2ms/step - loss: 130.8624 - mae: 11.1168 - mse: 130.8624 - mape: 686684.8750
Epoch 6/300
17/17 [==============================] - 0s 2ms/step - loss: 127.5227 - mae: 10.9473 - mse: 127.5227 - mape: 803534.0000
Epoch 7/300
17/17 [==============================] - 0s 2ms/step - loss: 124.0795 - mae: 10.7717 - mse: 124.0795 - mape: 960551.4375
Epoch 8/300
17/17 [==============================] - 0s 2ms/step - los

we can see that as accuracy increasing with the running epochs the loss is decresing.
we r able to make 86 correct predictions and 33 wrong predictions

## Part 4 - Making the predictions and evaluating the model

In [19]:
data2.head()

,G1,G2,G3,failures,studytime,freetime,absences,internet
0,0,11,11,0,2,3,4,no
1,9,11,11,0,2,3,2,yes
2,12,13,12,0,2,3,6,yes
3,14,14,14,0,3,2,0,yes
4,11,13,13,0,2,3,0,no


In [20]:
data2.describe()

,G1,G2,G3,failures,studytime,freetime,absences
count,649.000000,649.000000,649.000000,649.000000,649.000000,649.000000,649.000000
mean,11.399076,11.570108,11.906009,0.221880,1.930663,3.180277,3.659476
std,2.745265,2.913639,3.230656,0.593235,0.829510,1.051093,4.640759
min,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000
25%,10.000000,10.000000,10.000000,0.000000,1.000000,3.000000,0.000000
50%,11.000000,11.000000,12.000000,0.000000,2.000000,3.000000,2.000000
75%,13.000000,13.000000,14.000000,0.000000,2.000000,4.000000,6.000000
max,19.000000,19.000000,19.000000,3.000000,4.000000,5.000000,32.000000


In [29]:
x[3,:]

array([14, 14, 0, 3, 2, 0, 1], dtype=object)

In [30]:
# example_pred = (x[1,:])

# example_pred = np.asarray(example_pred).astype(np.int64)

example_res = ann.predict(sc.transform([[14, 14, 0, 3, 2, 0, 1]])).flatten()

example_res

array([13.176099], dtype=float32)

### Predicting the result of a single observation

**Homework**

Use our ANN model to predict if the customer with the following informations will leave the bank: 

Geography: France

Credit Score: 600

Gender: Male

Age: 40 years old

Tenure: 3 years

Balance: \$ 60000

Number of Products: 2

Does this customer have a credit card? Yes

Is this customer an Active Member: Yes

Estimated Salary: \$ 50000

So, should we say goodbye to that customer?

**Solution**

In [23]:
print(ann.predict(sc.transform([[7,3,9,6]])))

# the above will give the output interms of probability bw 0 to 1 including 
# as we have used the sigmoid function in the output layer 

# things to keep in mind 
# ---imp_1---
# include double square brakcets as the prdict method always expects 2D array as the format of input 
# ---imp_2---
# also the input for the parameter Geography expects a One-Hot-Encoded values 
# ---imp_3---
# predict method has to be called on the observations on which the same scaling has to be applied as it was applied on the test dataset
# as we trained our Model on the scaled test set the same scaling has to be applied to the obseervations that have to prdict the result
# it is always the case for neural networks and deep learning to apply scaling 
# fit_transform is not used and only transform as fit_transform is used to get the mean and standard deviation of the values and fit it to the training set
# so we need only transform method as if we apply fit agian then it will result in info leakage

# print(ann.predict(sc.transform([[1, 0, 0, 600, 1, 40, 3, 60000, 2, 1, 1, 50000]])) > 0.5)
# in order to print True or False the above comparison can be used 
# false meaning that the customer will not leave the bank

ValueError: ignored

Therefore, our ANN model predicts that this customer stays in the bank!

**Important note 1:** Notice that the values of the features were all input in a double pair of square brackets. That's because the "predict" method always expects a 2D array as the format of its inputs. And putting our values into a double pair of square brackets makes the input exactly a 2D array.

**Important note 2:** Notice also that the "France" country was not input as a string in the last column but as "1, 0, 0" in the first three columns. That's because of course the predict method expects the one-hot-encoded values of the state, and as we see in the first row of the matrix of features X, "France" was encoded as "1, 0, 0". And be careful to include these values in the first three columns, because the dummy variables are always created in the first columns.

### Predicting the Test set results

In [ ]:
# this is done to compare the y_pred and y_test respectively

y_pred = ann.predict(x_test)

y_pred = y_pred > 0.5

print(np.concatenate((y_pred.reshape(len(y_pred),1), y_test.reshape(len(y_test),1)),1))
# remember we used the same in NLP

### Making the Confusion Matrix

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

# 86.2% correct predictions 
# remember we used the same in NLP